# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Perlin Noise

In [2]:
import time

In [1]:
!pip install --upgrade bokeh==2.4.3

In [3]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import gray
from bokeh.layouts import layout

output_notebook()

# Algorithm

In [4]:
%%time
def generate_gradient(seed=None):
    global gradient

    seed and np.random.seed(seed)
    gradient = np.random.rand(512, 512, 2) * 2 - 1

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [5]:
%%time
def perlin_noise(size_x, size_y, frequency):
    global gradient

    # linear space by frequency
    x = np.tile(np.linspace(0, frequency, size_x, endpoint=False), size_y)
    y = np.repeat(np.linspace(0, frequency, size_y, endpoint=False), size_x)

    # gradient coordinates
    x0 = x.astype(int)
    y0 = y.astype(int)

    # local coordinate
    x -= x0
    y -= y0

    # gradient projections
    g00 = gradient[x0, y0]
    g10 = gradient[x0 + 1, y0]
    g01 = gradient[x0, y0 + 1]
    g11 = gradient[x0 + 1, y0 + 1]

    # fade
    t = (3 - 2 * x) * x * x

    # linear interpolation
    r = g00[:, 0] * x + g00[:, 1] * y
    s = g10[:, 0] * (x - 1) + g10[:, 1] * y
    g0 = r + t * (s - r)

    # linear interpolation
    r = g01[:, 0] * x + g01[:, 1] * (y - 1)
    s = g11[:, 0] * (x - 1) + g11[:, 1] * (y - 1)
    g1 = r + t * (s - r)

    # fade
    t = (3 - 2 * y) * y * y

    # (bi)linear interpolation
    g = g0 + t * (g1 - g0)

    # reshape
    return g.reshape(size_y, size_x)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.7 µs


In [6]:
%%time
def banded_perlin_noise(size_x, size_y, frequencies, amplitudes):
    image = np.zeros((size_y, size_x))

    for f, a in zip(frequencies, amplitudes):
        image += perlin_noise(size_x, size_y, f) * a

    image -= image.min()
    image /= image.max()

    return image

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 8.58 µs


# Run

## Perlin Noise

In [7]:
%%time
generate_gradient()

CPU times: user 8.2 ms, sys: 2.89 ms, total: 11.1 ms
Wall time: 16.9 ms


In [8]:
%%time
plots = []

for frequency in [1, 2, 4, 8, 16, 32, 64, 128]:
    image = perlin_noise(200, 200, frequency)

    plot = figure(x_range=(0, 1), y_range=(0, 1), plot_width=200, plot_height=200)
    plot.axis.visible = False
    plot.toolbar_location = None
    plot.min_border = 0
    plot.image([image], x=0, y=0, dw=1, dh=1, palette=gray(256))

    plots.append(plot)

show(layout([plots[:4], plots[4:]]))

Output hidden; open in https://colab.research.google.com to view.

## Perlin Noise

In [9]:
%%time
generate_gradient(39320)

CPU times: user 499 ms, sys: 22 ms, total: 521 ms
Wall time: 536 ms
CPU times: user 13.7 ms, sys: 85 µs, total: 13.8 ms
Wall time: 13.3 ms


In [10]:
%%time
image = banded_perlin_noise(512, 512, [2, 4, 8, 16, 32, 64], [32, 16, 8, 4, 2, 1])

CPU times: user 410 ms, sys: 98.5 ms, total: 509 ms
Wall time: 549 ms


In [11]:
%%time
start = np.array([0x42, 0x92, 0xc6])
end = np.array([0xf7, 0xfb, 0xff])
palette = [
    "#%02x%02x%02x" % tuple(((1 - i) * start + i * end).astype(int))
    for i in np.linspace(0, 1, num=256)
]

CPU times: user 3.76 ms, sys: 1.11 ms, total: 4.87 ms
Wall time: 5.47 ms


## Plot

In [12]:
%%time
plot = figure(x_range=(0, 1), y_range=(0, 1), plot_width=512, plot_height=512)

plot.axis.visible = False
plot.toolbar_location = None
plot.min_border = 0
plot.image([image], x=0, y=0, dw=1, dh=1, palette=palette)

show(plot)

Output hidden; open in https://colab.research.google.com to view.

# The End